In [ ]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import logging

In [ ]:

# Example input data
initial_state_tracts = gpd.read_file("../data/final_results.geojson")

# Set up logging
logger = logging.getLogger(__name__)

In [2]:
def create_buckets_from_tracts(
    initial_state_tracts: gpd.GeoDataFrame,
    geoid_field_name: str,
    target_score_field: str,
    high_low_zoom_threshold: int,
    number_of_buckets: int,
    homogeneity_threshold: int
):
    """
    Groups geographic tracts into buckets based on their scores.

    Args:
        initial_state_tracts (gpd.GeoDataFrame): GeoDataFrame containing the tracts.
        geoid_field_name (str): The name of the GEOID field in the GeoDataFrame.
        target_score_field (str): The name of the score field to use for bucketing.
        high_low_zoom_threshold (int): Minimum number of tracts required to avoid aggregation.
        number_of_buckets (int): Initial number of buckets to divide the tracts into.
        homogeneity_threshold (int): Threshold for adjusting bucket sizes for homogeneity.

    Returns:
        tuple: A tuple containing:
            - state_tracts (gpd.GeoDataFrame): Tracts grouped into buckets.
            - high_zoom_tracts (gpd.GeoDataFrame): Tracts kept at high zoom levels.
    """
    # Step 1: Identify states with fewer tracts than the threshold
    highzoom_state_tracts = initial_state_tracts.reset_index()
    highzoom_state_tracts["state"] = highzoom_state_tracts[geoid_field_name].astype(str).str[:2]
    keep_high_zoom = highzoom_state_tracts.groupby("state")[geoid_field_name].transform(
        lambda x: x.count() <= high_low_zoom_threshold
    )

    # Ensure some tracts are kept at high zoom
    assert keep_high_zoom.sum() != initial_state_tracts.shape[0], \
        "Error: Cutoff is too high, nothing is aggregated"
    assert keep_high_zoom.sum() > 1, "Error: Nothing is kept at high zoom"

    # Step 2: Separate tracts for high zoom and those to be bucketed
    state_tracts = initial_state_tracts[~keep_high_zoom].copy()
    state_tracts[f"{target_score_field}_bucket"] = np.arange(len(state_tracts))

    # Step 3: Sort tracts by score and calculate bucket size
    state_tracts = state_tracts.sort_values(target_score_field, ascending=True)
    score_bucket = []
    bucket_size = math.ceil(len(state_tracts.index) / number_of_buckets)

    # Step 4: Adjust bucket size for homogeneity
    while state_tracts[target_score_field].sum() % bucket_size > homogeneity_threshold:
        number_of_buckets += 1
        bucket_size = math.ceil(len(state_tracts.index) / number_of_buckets)

    logger.debug(f"The number of buckets has increased to {number_of_buckets}")

    # Step 5: Assign tracts to buckets
    for i in range(len(state_tracts.index)):
        score_bucket.append(math.floor(i / bucket_size))
    state_tracts[f"{target_score_field}_bucket"] = score_bucket

    # Step 6: Return bucketed tracts and high zoom tracts
    return state_tracts, initial_state_tracts[keep_high_zoom]

In [ ]:
# initial_state_tracts['tract_id'].astype(str).str[:2]

In [ ]:
# Parameters
geoid_field_name = "tract_id"
target_score_field = "dac"
high_low_zoom_threshold = 150
number_of_buckets = 10
homogeneity_threshold = 200

# Call the function
bucketed_tracts, high_zoom_tracts = create_buckets_from_tracts(
    initial_state_tracts,
    geoid_field_name,
    target_score_field,
    high_low_zoom_threshold,
    number_of_buckets,
    homogeneity_threshold
)


In [ ]:
# Output results
print("Bucketed Tracts:")
bucketed_tracts.head()


Bucketed Tracts:


,tract_id,county,state,black,american_indian_alaskan_native,asian,native_hawaiian_pacific_islander,two_or_more_races,white,hispanic_latino,...,total_burdens,dac,percent_area_dac,total_population,gis_sim_burd,p_sim_burd,gis_sim_ind,p_sim_ind,geometry,dac_bucket
0,1001020100,Autauga County,Alabama,0.07,0.0,0.00,0.0,0.07,0.83,0.01,...,0.0,False,0,1993.0,-1.237826,-0.0001,-1.123479,-0.0001,"POLYGON ((-86.48196 32.49876, -86.48189 32.498...",0
34138,26125157100,Oakland County,Michigan,0.20,0.0,0.08,0.0,0.06,0.63,0.00,...,0.0,False,0,1904.0,-1.509808,-0.0001,-1.379867,-0.0001,"POLYGON ((-83.36021 42.55755, -83.35776 42.557...",0
34139,26125157200,Oakland County,Michigan,0.21,0.0,0.04,0.0,0.02,0.70,0.01,...,0.0,False,0,3897.0,-1.631153,-0.0001,-1.488212,-0.0001,"POLYGON ((-83.42007 42.55094, -83.42006 42.550...",0
34140,26125157300,Oakland County,Michigan,0.20,0.0,0.20,0.0,0.00,0.57,0.01,...,0.0,False,0,3351.0,-1.631153,-0.0001,-1.488212,-0.0001,"POLYGON ((-83.43252 42.52628, -83.43264 42.526...",0
34141,26125157400,Oakland County,Michigan,0.12,0.0,0.09,0.0,0.02,0.75,0.00,...,0.0,False,0,2627.0,-1.746970,-0.0001,-1.586091,-0.0001,"POLYGON ((-83.41409 42.52682, -83.41606 42.526...",0


In [ ]:
print("High Zoom Tracts:")
high_zoom_tracts.head()

High Zoom Tracts:


,tract_id,county,state,black,american_indian_alaskan_native,asian,native_hawaiian_pacific_islander,two_or_more_races,white,hispanic_latino,...,total_criteria,total_burdens,dac,percent_area_dac,total_population,gis_sim_burd,p_sim_burd,gis_sim_ind,p_sim_ind,geometry
72607,56001962700,Albany County,Wyoming,0.00,0.00,0.01,0.0,0.02,0.91,0.07,...,0,0.0,False,0,3693.0,-0.159678,-0.4614,-0.083880,-0.4995,"POLYGON ((-105.73169 41.25981, -105.73168 41.2..."
72608,56001962800,Albany County,Wyoming,0.00,0.07,0.00,0.0,0.01,0.78,0.13,...,2,1.0,True,100,3126.0,0.306277,0.3895,0.211735,0.3519,"POLYGON ((-105.62483 41.30337, -105.62488 41.3..."
72609,56001962900,Albany County,Wyoming,0.00,0.00,0.00,0.0,0.09,0.59,0.37,...,4,3.0,True,100,1677.0,-0.641825,-0.2508,-0.475527,-0.3471,"POLYGON ((-105.59661 41.3122, -105.5968 41.311..."
72610,56001963000,Albany County,Wyoming,0.00,0.00,0.02,0.0,0.01,0.84,0.10,...,3,2.0,True,100,2482.0,-0.410606,-0.3381,-0.420086,-0.3841,"POLYGON ((-105.59125 41.3328, -105.59123 41.33..."
72611,56001963100,Albany County,Wyoming,0.02,0.00,0.03,0.0,0.03,0.82,0.07,...,0,0.0,False,0,8223.0,-0.889677,-0.1753,-0.793266,-0.2260,"POLYGON ((-105.58354 41.32749, -105.58348 41.3..."


In [7]:
def aggregate_buckets(
    state_tracts: gpd.GeoDataFrame,
    target_score_field: str,
    geometry_field_name: str,
    agg_func: str
) -> gpd.GeoDataFrame:
    """
    Aggregates tracts into buckets by dissolving geometries.

    Args:
        state_tracts (gpd.GeoDataFrame): GeoDataFrame containing tracts and their bucket assignments.
        target_score_field (str): The name of the score field.
        geometry_field_name (str): The name of the geometry field.
        agg_func (str): Aggregation function to use (e.g., "mean").

    Returns:
        gpd.GeoDataFrame: Aggregated GeoDataFrame.
    """
    keep_cols = [
        target_score_field,
        f"{target_score_field}_bucket",
        geometry_field_name,
    ]

    # Dissolve tracts by their bucket
    state_dissolve = state_tracts[keep_cols].dissolve(
        by=f"{target_score_field}_bucket", aggfunc=agg_func
    )
    return state_dissolve

In [10]:
# Parameters
target_score_field = "dac"
geometry_field_name = "geometry"
agg_func = "mean"
num_buckets = 10
state_tracts = bucketed_tracts.copy()

# Aggregate buckets
aggregated = aggregate_buckets(state_tracts, target_score_field, geometry_field_name, agg_func)

In [ ]:
aggregated.reset_index(inplace=True)


dac_bucket
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
Name: count, dtype: int64

In [21]:
aggregated.set_index("dac_bucket", inplace=True)

In [22]:
def breakup_multipolygons(
    state_bucketed_df: gpd.GeoDataFrame,
    target_score_field: str,
    geometry_field_name: str,
    num_buckets: int
) -> list:
    """
    Breaks up multipolygon geometries into individual polygons.

    Args:
        state_bucketed_df (gpd.GeoDataFrame): GeoDataFrame containing bucketed geometries.
        target_score_field (str): The name of the score field.
        geometry_field_name (str): The name of the geometry field.
        num_buckets (int): Number of buckets.

    Returns:
        list: A list of individual polygons with their associated scores.
    """
    compressed = []
    for i in range(num_buckets):
        for j in range(len(state_bucketed_df[geometry_field_name][i].geoms)):
            compressed.append(
                [
                    state_bucketed_df[target_score_field][i],
                    state_bucketed_df[geometry_field_name][i].geoms[j],
                ]
            )
    return compressed

In [23]:
# Break up multipolygons
compressed = breakup_multipolygons(aggregated, target_score_field, geometry_field_name, num_buckets)

In [29]:
def join_high_and_low_zoom_frames(
    compressed: list,
    keep_high_zoom_df: gpd.GeoDataFrame,
    target_score_field: str,
    geometry_field_name: str
) -> gpd.GeoDataFrame:
    """
    Combines high-zoom tracts with bucketed low-zoom tracts.

    Args:
        compressed (list): List of individual polygons with their scores.
        keep_high_zoom_df (gpd.GeoDataFrame): GeoDataFrame of high-zoom tracts.
        target_score_field (str): The name of the score field.
        geometry_field_name (str): The name of the geometry field.

    Returns:
        gpd.GeoDataFrame: Combined GeoDataFrame.
    """
    keep_columns = [
        target_score_field,
        geometry_field_name,
    ]
    compressed_geodf = gpd.GeoDataFrame(
        compressed,
        columns=keep_columns,
        crs="EPSG:4326",
    )
    return pd.concat([compressed_geodf, keep_high_zoom_df[keep_columns]])

In [30]:
# Combine high-zoom and low-zoom frames
keep_high_zoom_df = initial_state_tracts.copy()
combined = join_high_and_low_zoom_frames(compressed, keep_high_zoom_df, target_score_field, geometry_field_name)

In [35]:
initial_state_tracts['dac'].value_counts()

dac
False    45199
True     28568
Name: count, dtype: int64

In [37]:
28568/(45199 + 28568)

0.387273442054035

In [38]:
combined['dac'].value_counts()

dac
0.000000    46663
1.000000    30077
0.032178      296
Name: count, dtype: int64

In [ ]:

# Parameters
target_score_field = "dac"
geometry_field_name = "geometry"
agg_func = "mean"
num_buckets = 10

# Aggregate buckets
aggregated = aggregate_buckets(state_tracts, target_score_field, geometry_field_name, agg_func)

# Break up multipolygons
compressed = breakup_multipolygons(aggregated, target_score_field, geometry_field_name, num_buckets)

# Combine high-zoom and low-zoom frames
keep_high_zoom_df = gpd.read_file("path_to_high_zoom_geojson.geojson")
combined = join_high_and_low_zoom_frames(compressed, keep_high_zoom_df, target_score_field, geometry_field_name)


In [39]:
# Save the result
combined.to_file("../data/usa_low.geojson", driver="GeoJSON")